In [1]:
import pandas as pd
import jsonpickle
import numpy as np
import matplotlib.pyplot as plt
import os, os.path
import dateutil.parser

In [2]:
FILE_AS_ROOT = True
FILE_TO_ROOT = '../../../../../../'
JSON_FILE_TO_IMPORT = f"{ FILE_TO_ROOT if FILE_AS_ROOT else '../'}tagging_data_personal.json" 
SCRIPT_PATH = "./src/python/psic/stats/tagging_stats"
FILE_TO_TAG_RATIO = f"{'./' if FILE_AS_ROOT else './src/python/psic/stats/tagging_stats'}tag_ratio.csv"
TAGGING_STATES_FOLDER = f"{ FILE_TO_ROOT if FILE_AS_ROOT else '../'}tagging_json_states"

In [3]:
TOTAL_JSON_STATES = (len([name for name in os.listdir(TAGGING_STATES_FOLDER) if os.path.isfile(os.path.join(TAGGING_STATES_FOLDER, name))]))
TOTAL_JSON_STATES

892

In [4]:
def get_pickle(path_to_file):
    with open(path_to_file, 'r') as f:
        return jsonpickle.decode(f.read())

In [5]:
def how_many_tagged(pickle_file):
    done_tagging_count=len(pickle_file.finished_tagged_queue)
    tagged_but_not_done_count=0
    for image in pickle_file.pending_images_queue:
        if len(image.get_taggers()) > 0:
            tagged_but_not_done_count-=-1
    
#     print(f"{tagged_but_not_done_count},{done_tagging_count}")

#     print(f"{tagged_but_not_done_count} images have been tagged, but not completely")
#     print(f"{done_tagging_count} images have been tagged completely")
#     print(f"{tagged_but_not_done_count+done_tagging_count} images have been tagged, completely or not")
    
    return pd.DataFrame([{ 
        'not_done':tagged_but_not_done_count,
        'done':done_tagging_count,
        'tagged_ratio': tagged_but_not_done_count/(tagged_but_not_done_count+done_tagging_count)
     }])

In [6]:
path = os.getcwd()
files = os.listdir(f"{FILE_TO_ROOT}tagging_json_states")
files_json = [f for f in files if f[-4:] == 'json']
files_json.sort()

In [12]:
def get_tagged_ratio(files):
    main_df = pd.DataFrame()
    for file in files[::50]:
        path = f"{FILE_TO_ROOT}tagging_json_states/{file}"
        
        remove_name = file.split('assigner_state-root-')[-1]
        date_string = remove_name.split('.json')[0]
        removed_nano = date_string.split('.')[0].replace("_",":")
        time = dateutil.parser.parse(removed_nano).strftime('%s')
        time_df= pd.DataFrame([{'time':time}])    
        
        tag_df = how_many_tagged(get_pickle(path))
        
        combined_df = pd.concat([time_df,tag_df],axis=1, sort=False)
        main_df = pd.concat([combined_df,main_df])
    return main_df
    
df = get_tagged_ratio(files_json).sort_values(by=['time'])

KeyError: 'time'

In [ ]:
type(df[['tagged_ratio']])
